Разделяем данные на тренировочные, тестовые и валидационные

In [1]:
import os
import shutil
import random

# Путь к папке с изображениями и аннотациями
data_dir = "dataset/agri_data/data"
output_dir = "dataset/agri_data"

# Создание папок для разделенных данных
for folder in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_dir, folder, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, folder, "labels"), exist_ok=True)

# Получение списка всех файлов изображений
images = [f for f in os.listdir(data_dir) if f.endswith(".jpeg")]

# Перемешиваем список изображений и разделяем
random.shuffle(images)
train_split = int(0.8 * len(images))
val_split = int(0.9 * len(images))

train_images = images[:train_split]
val_images = images[train_split:val_split]
test_images = images[val_split:]

# Функция для копирования изображений и аннотаций
def copy_files(image_list, split):
    for image in image_list:
        label = image.replace(".jpeg", ".txt")
        shutil.copy(os.path.join(data_dir, image), os.path.join(output_dir, split, "images", image))
        shutil.copy(os.path.join(data_dir, label), os.path.join(output_dir, split, "labels", label))

# Копируем файлы
copy_files(train_images, "train")
copy_files(val_images, "val")
copy_files(test_images, "test")

print("Данные успешно разделены на тренировочные, валидационные и тестовые.")


Данные успешно разделены на тренировочные, валидационные и тестовые.


1. Создание конфигурационного файла data.yaml

In [3]:
# Пути к вашим папкам с изображениями и аннотациями
train_images_path = 'dataset/train/images/'
val_images_path = 'dataset/val/images/'
test_images_path = 'dataset/test/images/'

# Создаем data.yaml
data_yaml = f"""
train: {train_images_path}  # Путь к тренировочным изображениям
val: {val_images_path}      # Путь к валидационным изображениям
test: {test_images_path}    # Путь к тестовым изображениям

nc: 2                       # Число классов
names: ['crop', 'weed']     # Названия классов
"""

# Запись конфигурации в файл data.yaml
with open('data.yaml', 'w') as f:
    f.write(data_yaml)


2. Настройка гиперпараметров и запуск обучения

In [1]:
from ultralytics import YOLO

# Загрузка предобученной модели YOLOv11
model = YOLO("yolo11m.pt")

# Запуск обучения
train_results = model.train(
    data="data.yaml",  # Путь к конфигурационному файлу данных
    epochs=50,         # Количество эпох
    imgsz=640,         # Размер изображений
    batch=16,          # Размер батча
    device=0           # Указание GPU (0 — первый GPU, или "cpu" для процессора)
)



Ultralytics 8.3.31  Python-3.12.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11m.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

100%|███████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 4.10MB/s]

Overriding model.yaml nc=80 with nc=2



                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512

100%|█████████████████████████████████████████████████████████████████████████████| 5.35M/5.35M [00:00<00:00, 16.1MB/s]


...

      50/50      8.56G     0.9485     0.6653       1.45         19        640: 100%|██████████| 65/65 [00:44<00:00,  1.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:06<0

                   all        130        233      0.771      0.843       0.85      0.565



50 epochs completed in 0.719 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 40.5MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 40.5MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.31  Python-3.12.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
YOLO11m summary (fused): 303 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<0


                   all        130        233      0.772      0.836       0.85      0.565
                  crop         71        154      0.682      0.805      0.831      0.552
                  weed         60         79      0.861      0.866      0.869      0.579
Speed: 0.2ms preprocess, 14.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train2


3. Оценка модели на валидационном наборе

In [11]:
# Оценка производительности модели на валидационном наборе с указанием data.yaml
metrics = model.val(data="data.yaml")
print(metrics)  # Вывод метрик, таких как mAP и другие показатели

Ultralytics 8.3.31  Python-3.12.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)


val: Scanning E:\progs\AgroVision\dataset\labels\val.cache... 130 images, 0 backgrounds, 0 corrupt: 100%|██████████| 130/130 [00:00<?,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:06<00:00,  1.30it/s


                   all        130        233      0.771      0.843       0.85      0.566
                  crop         71        154       0.68      0.818      0.831      0.552
                  weed         60         79      0.862      0.868       0.87      0.579
Speed: 1.0ms preprocess, 10.9ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002D717C97020>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,  

In [10]:
# Загрузка обученной модели
model = YOLO("runs/detect/train2/weights/best.pt")  # Используйте путь к файлу весов, который сохранился после обучения

# Запуск инференса на изображениях в папке test
results = model.predict(
    source="E:/progs/AgroVision/dataset/images/test",  # Путь к тестовым изображениям
    conf=0.25,                                         # Порог уверенности
    imgsz=640,                                         # Размер изображения
    save=True                                          # Сохраняет результаты, не выводя их на экран
)



image 1/130 E:\progs\AgroVision\dataset\images\test\agri_0_1024.jpeg: 640x640 1 weed, 21.9ms
image 2/130 E:\progs\AgroVision\dataset\images\test\agri_0_1026.jpeg: 640x640 1 crop, 23.9ms
image 3/130 E:\progs\AgroVision\dataset\images\test\agri_0_106.jpeg: 640x640 1 weed, 15.0ms
image 4/130 E:\progs\AgroVision\dataset\images\test\agri_0_1068.jpeg: 640x640 2 weeds, 19.9ms
image 5/130 E:\progs\AgroVision\dataset\images\test\agri_0_1146.jpeg: 640x640 1 weed, 15.0ms
image 6/130 E:\progs\AgroVision\dataset\images\test\agri_0_1171.jpeg: 640x640 1 crop, 15.0ms
image 7/130 E:\progs\AgroVision\dataset\images\test\agri_0_1350.jpeg: 640x640 1 weed, 15.0ms
image 8/130 E:\progs\AgroVision\dataset\images\test\agri_0_1385.jpeg: 640x640 1 weed, 15.0ms
image 9/130 E:\progs\AgroVision\dataset\images\test\agri_0_1463.jpeg: 640x640 1 crop, 15.0ms
image 10/130 E:\progs\AgroVision\dataset\images\test\agri_0_1518.jpeg: 640x640 2 weeds, 15.0ms
image 11/130 E:\progs\AgroVision\dataset\images\test\agri_0_1693.jp